In [ ]:
import os
%pwd

In [2]:
os.chdir("../")

In [ ]:
%pwd

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataEmbeddingConfig:
    root_dir: Path
    data_path: Path
    model_name: str
    text_column: str

@dataclass(frozen=True)
class VectorStorageConfig:
    data_path: Path
    embedding_dim: int

@dataclass
class GeminiConfig:
    # Gemini API 設定
    gemini_api_key: str
    gemini_model: str
    gemini_temperature: float
    gemini_max_output_tokens: int
    
    def __post_init__(self):
        if not self.gemini_api_key:
            raise ValueError("請設定 GEMINI_API_KEY 環境變數或在配置中提供 API key")
        
@dataclass
class RAGSystemConfig:
    max_retrieval_docs: int

In [ ]:
from dialogue_rag_chatbot.constants import *
from dialogue_rag_chatbot.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_embedding_config(self) -> DataEmbeddingConfig:
        config = self.config.data_embedding

        create_directories([config.root_dir])
        data_embedding_config = DataEmbeddingConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_name = config.model_name,
            text_column= config.text_column
        )

        return data_embedding_config

    def get_vector_storage_config(self)-> VectorStorageConfig:

        config = self.config.vector_storage
        vector_storage_config = VectorStorageConfig(
            data_path=config.data_path,
            embedding_dim=config.embedding_dim
        )

        return vector_storage_config
    
    def get_gemini_config(self)-> GeminiConfig:
        config = self.config.gemini
        gemini_config = GeminiConfig(
            gemini_api_key=os.getenv("GEMINI_API_KEY", ""),
            gemini_model=config.gemini_model,
            gemini_temperature= config.gemini_temperature,
            gemini_max_output_tokens=config.gemini_max_output_tokens
        )

        return gemini_config
    
    def get_rag_sys_config(self)-> RAGSystemConfig:
        config = self.config.RAG_system
        rag_sys_config = RAGSystemConfig(
            max_retrieval_docs = config.max_retrieval_docs
        )

        return rag_sys_config

In [ ]:
from dialogue_rag_chatbot.logging import logger

from sentence_transformers import SentenceTransformer
from datasets import load_from_disk
from typing import List, Dict, Any, Tuple
import numpy as np

from google import genai
from google.genai import types
from enum import Enum

from datetime import datetime
import time

c:\Users\Kun\miniconda3\envs\legalrag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class RetrieveDecision(str, Enum):
    """檢索決策枚舉"""
    YES = "yes"
    NO = "no" 
    CONTINUE = "continue"

class IsREL(str, Enum):
    """相關性判斷枚舉"""
    RELEVANT = "relevant"
    IRRELEVANT = "irrelevant"

class IsSUP(str, Enum):
    """支撐性判斷枚舉"""
    FULLY_SUPPORTED = "fully supported"
    PARTIALLY_SUPPORTED = "partially supported"
    NO_SUPPORT = "no support"

class IsUSE(int, Enum):
    """有用性評分枚舉"""
    VERY_USEFUL = 5
    USEFUL = 4
    MODERATELY_USEFUL = 3
    LESS_USEFUL = 2
    NOT_USEFUL = 1

In [10]:
class EmbeddingModel:
    """Granite embedding model using SentenceTransformers"""
    
    def __init__(self, config: DataEmbeddingConfig):
        self.model_name = config.model_name
        try:
            self.model = SentenceTransformer(self.model_name)
            logger.info(f"Granite embedding model '{self.model_name}' loaded successfully")
        except Exception as e:
            logger.error(f"Failed to load model {self.model_name}: {str(e)}")
            raise
    
    def encode(self, texts):
        """Encode texts into embeddings"""
        try:
            # Granite embedding models return 768-dimensional vectors
            embeddings = self.model.encode(texts, convert_to_numpy=True)
            logger.info(f"Encoded {len(texts)} texts into embeddings of shape {embeddings.shape}")
            return embeddings
        except Exception as e:
            logger.error(f"Error encoding texts: {str(e)}")
            raise


class VectorStorage:
    """Store and retrieve document embeddings"""
    
    def __init__(self, config: VectorStorageConfig):
        self.config = config
        self.embedding_dim = self.config.embedding_dim
        self.embeddings = []
        self.documents = []
        logger.info(f"VectorStore initialized with embedding_dim={self.embedding_dim}")
    
    def add_documents(self, documents: List[Dict[str, Any]], embeddings: np.ndarray):
        """Add documents and their embeddings to the store"""
        if embeddings.shape[1] != self.embedding_dim:
            raise ValueError(f"Embedding dimension mismatch: expected {self.embedding_dim}, got {embeddings.shape[1]}")
        
        self.embeddings.extend(embeddings.tolist())
        self.documents.extend(documents)
        logger.info(f"Added {len(documents)} documents to vector store")
    
    def similarity_search(self, query_embedding: np.ndarray, top_k: int = 5) -> List[Tuple[Dict[str, Any], float]]:
        """Perform similarity search"""
        if not self.embeddings:
            return []
        
        query_embedding = query_embedding.flatten()
        
        embeddings_matrix = np.array(self.embeddings)
        similarities = np.dot(embeddings_matrix, query_embedding.T) / (
            np.linalg.norm(embeddings_matrix, axis=1) * np.linalg.norm(query_embedding) + 1e-10
        )
        
        # top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        top_indices = np.argpartition(-similarities, top_k)[:top_k]
        top_indices = top_indices[np.argsort(-similarities[top_indices])]

        results = [(self.documents[idx], float(similarities[idx])) for idx in top_indices]
        logger.info(f"Retrieved {len(results)} documents from vector store")
        return results

In [11]:
class GeminiClient:
    """Gemini API 客戶端封裝"""
    
    def __init__(self, config: GeminiConfig):
        self.config = config
        self.client = genai.Client(api_key=config.gemini_api_key)
        logger.info(f"Gemini 客戶端初始化完成，使用模型: {config.gemini_model}")
    
    def generate_content(self, prompt: str, context: str = "") -> str:
        """生成文本內容"""
        try:
            full_prompt = f"{context}\n\n{prompt}" if context else prompt

            response = self.client.models.generate_content(
                model=self.config.gemini_model,
                contents=full_prompt,
                config=types.GenerateContentConfig(
                    temperature=self.config.gemini_temperature,
                    max_output_tokens=self.config.gemini_max_output_tokens
                )
            )
            
            print(f"產生內容===>: {response.candidates[0].content.parts[0].text}")
            
            if response and hasattr(response, 'text') and response.text:
                return response.text.strip()
            else:
                # 如果因為安全設定等原因沒有文字返回，則返回空字串
                logger.warning(f"Gemini API did not return text. Full response: {response}")
                return ""
            
        except Exception as e:
            logger.error(f"Gemini API 呼叫失敗: {str(e)}")
            return "抱歉，系統暫時無法處理您的請求。"
    
    def predict_retrieve(self, query: str, previous_generation: str = "") -> RetrieveDecision:
        """
        M predicts Retrieve given (x, y_{t-1})
        判斷是否需要檢索對話資料來回答問題
        """
        prompt = f"""
                    請判斷以下使用者查詢是否需要檢索對話資料庫來回答：

                    目前使用者查詢: {query}
                    之前的對話內容: {previous_generation if previous_generation else "無"}

                    判斷標準：
                    - yes: 查詢需要具體的對話內容或對話場景來回答。如果有之前的對話內容，請務必也要引用
                    - no: 查詢是一般性問題，可以直接回答，不需要特定對話內容

                    只允許回答: yes/no
                    """
        response = self.generate_content(prompt).lower().strip()
        if "yes" in response:
            return RetrieveDecision.YES
        else:
            return RetrieveDecision.NO
    
    def predict_isrel(self, query: str, dialogue: str) -> IsREL:
        """
        M predicts IsREL given x, d
        判斷對話是否與查詢相關
        """
        prompt = f"""
                請判斷以下對話內容是否與使用者查詢相關：

                使用者查詢: {query}
                對話內容: {dialogue}...

                判斷標準：
                - relevant: 對話包含與查詢直接相關的資訊、情境或主題
                - irrelevant: 對話與查詢無關或關聯性極低

                只允許回答: relevant/irrelevant
                """
        response = self.generate_content(prompt).lower().strip()
        return IsREL.RELEVANT if "relevant" in response else IsREL.IRRELEVANT
    
    def predict_issup(self, query: str, dialogue: str, candidate_answer: str) -> IsSUP:
        """
        M predicts IsSUP given x, y_t, d
        判斷對話是否支撐候選答案
        """
        prompt = f"""
                    請判斷以下對話內容是否支撐候選答案中的陳述：

                    使用者查詢: {query}
                    候選答案: {candidate_answer}
                    對話內容: {dialogue}...

                    判斷標準：
                    - fully supported: 答案中的陳述完全可在對話中找到依據
                    - partially supported: 部分陳述有依據，部分沒有
                    - no support: 答案沒有對話依據

                    只允許回答: fully supported/partially supported/no support
                """
        response = self.generate_content(prompt).lower().strip()
        if "fully supported" in response:
            return IsSUP.FULLY_SUPPORTED
        elif "partially supported" in response:
            return IsSUP.PARTIALLY_SUPPORTED
        else:
            return IsSUP.NO_SUPPORT
    
    def predict_isuse(self, query: str, candidate_answer: str, dialogue: str = "") -> IsUSE:
        """
        M predicts IsUSE given x, y_t, d
        評估候選答案的有用性
        """
        context = f"參考對話: {dialogue}..." if dialogue else ""
        prompt = f"""
                    請評估以下候選答案對使用者查詢的有用性(1-5分):

                    使用者查詢: {query}
                    候選答案: {candidate_answer}
                    {context}

                    評分標準：
                    5分 - 非常有用：完整回答問題，提供具體相關資訊
                    4分 - 有用：回答相關且有幫助
                    3分 - 中等：部分相關但不夠詳細
                    2分 - 較少用：相關性低或幫助有限
                    1分 - 無用：不相關或誤導性資訊

                    只允許回答數字: 1-5
                """
        response = self.generate_content(prompt).strip()
        try:
            score = int(response)
            return IsUSE(score) if 1 <= score <= 5 else IsUSE.MODERATELY_USEFUL
        except:
            return IsUSE.MODERATELY_USEFUL

In [12]:
class DialogueRAGSystem:
    """基於 SAMSum 對話資料集的 RAG 系統"""

    def __init__(self, config: ConfigurationManager):
        self.config = config
        self.embedding_model_config = self.config.get_data_embedding_config()
        self.embedding_model = EmbeddingModel(config= self.embedding_model_config)
        
        self.vector_storage_config = self.config.get_vector_storage_config()
        self.gemini_config = self.config.get_gemini_config()
        self.rag_sys_config = self.config.get_rag_sys_config()

        self.vector_store = VectorStorage(config=self.vector_storage_config)
        self.dataset_with_embeddings = load_from_disk(self.vector_storage_config.data_path)
        self.gemini_client = GeminiClient(config=self.gemini_config)
        self.config.max_retrieval_docs = self.rag_sys_config.max_retrieval_docs
        
        # 系統統計
        self.stats = {
            "total_queries": 0,
            "retrieval_queries": 0,
            "non_retrieval_queries": 0,
            "start_time": datetime.now()
        }

        logger.info("Dialogue RAG System 初始化完成")

    def build_knowledge_base(self):
        """建立對話知識庫"""
        logger.info("開始建立對話知識庫...")

        # 載入和處理對話
        documents = [{"id": item["id"], "dialogue": item["dialogue"], "summary": item["summary"]} for item in self.dataset_with_embeddings]
        embeddings = np.array(self.dataset_with_embeddings["embedding"])
        
        # 添加到向量存儲
        self.vector_store.add_documents(documents, embeddings)

    def query(self, user_query: str, conversation_history: List[str] = None) -> Dict[str, Any]:
        """
        主要查詢函數 - 實作完整的對話檢索流程
        """
        start_time = time.time()
        self.stats["total_queries"] += 1

        # Step 1: M predicts Retrieve given (x, y_{t-1})
        previous_generation = "\n".join(conversation_history[-3:]) if conversation_history else ""
        retrieve_decision = self.gemini_client.predict_retrieve(user_query, previous_generation)
        

        if retrieve_decision == RetrieveDecision.YES:
            return self._handle_retrieval_branch(user_query, previous_generation, start_time)
        else:
            return self._handle_non_retrieval_branch(user_query, start_time)

    def _handle_retrieval_branch(self, query: str, previous_generation: str, start_time: float) -> Dict[str, Any]:
        """處理需要檢索的分支"""
        self.stats["retrieval_queries"] += 1

        # Step 4: 檢索相關對話
        query_embedding = self.embedding_model.encode([query])[0]
        retrieved_docs = self.vector_store.similarity_search(
            query_embedding,
            top_k=self.rag_sys_config.max_retrieval_docs
        )

        if not retrieved_docs:
            return {
                "answer": "抱歉，我找不到相關的對話內容來回答您的問題。",
                "retrieve_decision": RetrieveDecision.YES.value,
                "sources": [],
                "processing_time": time.time() - start_time
            }

        # Step 5-7: 為每個相關對話生成候選答案並評估
        candidates = []

        for doc, score in retrieved_docs:
            # 判斷相關性
            
            relevance = self.gemini_client.predict_isrel(query, doc['dialogue'])
            # print(f"predict_isrel ====> {relevance}")
            if relevance == IsREL.RELEVANT:
                # 生成候選答案
                candidate_answer = self._generate_candidate_answer(query, doc, previous_generation)

                # 評估支撐性和有用性
                support_level = self.gemini_client.predict_issup(query, doc['dialogue'], candidate_answer)
                usefulness = self.gemini_client.predict_isuse(query, candidate_answer, doc['dialogue'])

                candidates.append({
                    'answer': candidate_answer,
                    'source_doc': doc,
                    'is_relevant': relevance,
                    'support_level': support_level,
                    'usefulness_score': usefulness
                })

        if not candidates:
            return {
                "answer": "檢索到的對話內容與您的問題不太相關，無法提供有效回答。",
                "retrieve_decision": RetrieveDecision.YES.value,
                "sources": [doc['id'] for doc, score in retrieved_docs],
                "processing_time": time.time() - start_time
            }

        # Step 8: 選擇最佳候選答案
        best_candidate = self._rank_candidates(candidates)

        return {
            "answer": best_candidate['answer'],
            "retrieve_decision": RetrieveDecision.YES.value,
            "sources": [best_candidate['source_doc']['id']],
            "relevance": best_candidate['is_relevant'].value,
            "support_level": best_candidate['support_level'].value,
            "usefulness_score": best_candidate['usefulness_score'].value,
            "processing_time": time.time() - start_time,
            "reference_dialogue": best_candidate['source_doc']['dialogue'][:300] + "..."
        }

    def _handle_non_retrieval_branch(self, query: str, start_time: float) -> Dict[str, Any]:
        """處理不需要檢索的分支"""
        self.stats["non_retrieval_queries"] += 1

        # Step 9: 直接生成答案
        generated_answer = self._generate_direct_answer(query)

        # Step 10: 評估有用性
        usefulness_score = self.gemini_client.predict_isuse(query, generated_answer)

        return {
            "answer": generated_answer,
            "retrieve_decision": RetrieveDecision.NO.value,
            "sources": [],
            "usefulness_score": usefulness_score.value,
            "processing_time": time.time() - start_time
        }

    def _generate_candidate_answer(self, query: str, dialogue_doc: Dict[str, Any], previous_generation: str) -> str:
        """為特定對話生成候選答案"""
        context = f"請基於以下對話內容回答使用者問題：\n\n對話內容: {dialogue_doc['dialogue']}\n對話摘要: {dialogue_doc['summary']}"

        if previous_generation:
            context += f"\n\n之前的對話: {previous_generation}"

        prompt = f"""
                    {context}

                    使用者問題: {query}

                    請提供有用的回答，並：
                    1. 直接回答使用者問題
                    2. 引用或描述相關的對話內容
                    3. 提供具體的資訊或見解
                    4. 保持回答簡潔明確
                """
        return self.gemini_client.generate_content(prompt)

    def _generate_direct_answer(self, query: str) -> str:
        """直接生成答案（不使用檢索）"""
        prompt = f"""
                    作為對話理解助手，請回答以下問題：

                    {query}

                    請基於一般知識提供回答，並：
                    1. 直接回答問題
                    2. 提供相關的背景資訊
                    3. 保持回答有用且相關
                """
        return self.gemini_client.generate_content(prompt)

    def _rank_candidates(self, candidates: List[Dict[str, Any]]) -> Dict[str, Any]:
        """根據 IsREL, IsSUP, IsUSE 排序候選答案"""
        def calculate_score(candidate):
            score = 0

            # IsREL 權重
            if candidate['is_relevant'] == IsREL.RELEVANT:
                score += 10

            # IsSUP 權重
            support_scores = {
                IsSUP.FULLY_SUPPORTED: 10,
                IsSUP.PARTIALLY_SUPPORTED: 5,
                IsSUP.NO_SUPPORT: 0
            }
            score += support_scores.get(candidate['support_level'], 0)

            # IsUSE 權重
            score += candidate['usefulness_score'].value * 2

            return score

        # 按分數排序
        sorted_candidates = sorted(candidates, key=calculate_score, reverse=True)
        return sorted_candidates[0]

    def get_system_stats(self) -> Dict[str, Any]:
        """獲取系統統計資訊"""
        uptime = datetime.now() - self.stats["start_time"]
        return {
            "total_queries": self.stats["total_queries"],
            "retrieval_queries": self.stats["retrieval_queries"],
            "non_retrieval_queries": self.stats["non_retrieval_queries"],
            "retrieval_rate": self.stats["retrieval_queries"] / max(1, self.stats["total_queries"]),
            "uptime_hours": uptime.total_seconds() / 3600,
            "dialogues_in_kb": len(self.vector_store.documents)
        }

In [13]:

try:
    config = ConfigurationManager()
    rag_system = DialogueRAGSystem(config)
    rag_system.build_knowledge_base()
        
except Exception as e:
    raise e

[2025-09-28 09:47:42,337: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-28 09:47:42,341: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-28 09:47:42,343: INFO: common: created directory at: artifacts]
[2025-09-28 09:47:42,344: INFO: common: created directory at: artifacts/data_embedding]
[2025-09-28 09:47:42,351: INFO: SentenceTransformer: Use pytorch device_name: cpu]
[2025-09-28 09:47:42,352: INFO: SentenceTransformer: Load pretrained SentenceTransformer: ibm-granite/granite-embedding-278m-multilingual]
[2025-09-28 09:47:48,834: INFO: 1956350838: Granite embedding model 'ibm-granite/granite-embedding-278m-multilingual' loaded successfully]
[2025-09-28 09:47:48,835: INFO: 1956350838: VectorStore initialized with embedding_dim=768]
[2025-09-28 09:47:49,796: INFO: 2269672936: Gemini 客戶端初始化完成，使用模型: gemini-2.0-flash]
[2025-09-28 09:47:49,797: INFO: 2765487390: Dialogue RAG System 初始化完成]
[2025-09-28 09:47:49,798: INFO: 2765487390: 開始建立對話知識庫...

In [14]:
try:
    test_query = "哪些人談話跟工作有關?"
    result = rag_system.query(test_query)
    
    print(f"🤖 回答: {result['answer']}")
    print(f"📊 檢索決策: {result['retrieve_decision']}")
    print(f"📁 參考來源: {result.get('sources', [])}")
    print(f"⏱️ 處理時間: {result['processing_time']:.2f}秒")

    if 'support_level' in result:
        print(f"🎯 支撐程度: {result['support_level']}")
    if 'usefulness_score' in result:
        print(f"⭐ 有用性評分: {result['usefulness_score']}/5")
    if 'reference_dialogue' in result:
        print(f"💬 參考對話: {result['reference_dialogue']}")

    # 顯示系統統計
    print(f"\n{'='*60}")
    print("📈 系統統計資訊")
    print(f"{'='*60}")
    stats = rag_system.get_system_stats()
    for key, value in stats.items():
        print(f"{key}: {value}")
except Exception as e:
    raise e

[2025-09-28 09:48:10,892: INFO: models: AFC is enabled with max remote calls: 10.]


[2025-09-28 09:48:11,817: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: yes



Batches: 100%|██████████| 1/1 [00:00<00:00,  7.15it/s]

[2025-09-28 09:48:11,980: INFO: 1956350838: Encoded 1 texts into embeddings of shape (1, 768)]


[2025-09-28 09:48:12,526: INFO: 1956350838: Retrieved 3 documents from vector store]
[2025-09-28 09:48:12,537: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:13,092: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: relevant

[2025-09-28 09:48:13,097: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:16,963: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: 以下是回答：

所有參與對話的人都談論了與工作相關的事情。他們分享了各自工作中最好的部分。例如：

*   **Alan** 提到了 "training opportunities" (培訓機會)。
*   **Tim** 說 "it's the people that i work with" (與我共事的人)。
*   **Harry** 說 "it is challenging so i learn a lot" (它具有挑戰性，所以我學到很多)。
*   **Bill** 提到 "flexibility! can work different hours" (彈性！可以工作不同的時間)。
*   **Jeff** 說 "free tea & coffee, fruit and lunch once a week" (免費茶和咖啡，水果和每週一次的午

In [15]:
result['answer']

'以下是回答：\n\n所有參與對話的人都談論了與工作相關的事情。他們分享了各自工作中最好的部分。例如：\n\n*   **Alan** 提到了 "training opportunities" (培訓機會)。\n*   **Tim** 說 "it\'s the people that i work with" (與我共事的人)。\n*   **Harry** 說 "it is challenging so i learn a lot" (它具有挑戰性，所以我學到很多)。\n*   **Bill** 提到 "flexibility! can work different hours" (彈性！可以工作不同的時間)。\n*   **Jeff** 說 "free tea & coffee, fruit and lunch once a week" (免費茶和咖啡，水果和每週一次的午餐)。\n*   **Sarah** 說 "that i don\'t have to wear posh clothes" (我不需要穿華麗的衣服)。\n*   **Eric** 說 "can work from home 3 days a week so save lots of time and money on commuting" (每週可以居家工作三天，所以節省了大量的通勤時間和金錢)。\n*   **Rob** 提到 "my salary" (我的薪水) 和 "it is near my house so i can walk to work" (它離我家很近，所以我可以走路去上班)。\n*   **Karen** 說 "passionate people around" (周圍有充滿熱情的人)。\n*   **Freddie** 說 "great manager! i also do stuff that i care about!" (很棒的經理！我也做我關心的事情！)。\n*   **Jamie** 說 "my favourite part of my job is that it\'s very creative" (我最喜歡的工作部分是非常有創意)。\n\n因此，所有參與者都在討論與他們工作相關的優點。'

In [16]:
try:
    test_query = "我想更進一步知道，誰提到跟'salary'有關?"
    res = rag_system.query(test_query, [result['answer']])
    
    print(f"🤖 回答: {res['answer']}")
    print(f"📊 檢索決策: {res['retrieve_decision']}")
    print(f"📁 參考來源: {res.get('sources', [])}")
    print(f"⏱️ 處理時間: {res['processing_time']:.2f}秒")

    if 'support_level' in res:
        print(f"🎯 支撐程度: {res['support_level']}")
    if 'usefulness_score' in res:
        print(f"⭐ 有用性評分: {res['usefulness_score']}/5")
    if 'reference_dialogue' in res:
        print(f"💬 參考對話: {res['reference_dialogue']}")

    # 顯示系統統計
    print(f"\n{'='*60}")
    print("📈 系統統計資訊")
    print(f"{'='*60}")
    stats = rag_system.get_system_stats()
    for key, value in stats.items():
        print(f"{key}: {value}")
except Exception as e:
    raise e

[2025-09-28 09:48:24,807: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:25,521: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: yes



Batches: 100%|██████████| 1/1 [00:00<00:00, 11.48it/s]

[2025-09-28 09:48:25,628: INFO: 1956350838: Encoded 1 texts into embeddings of shape (1, 768)]


[2025-09-28 09:48:26,162: INFO: 1956350838: Retrieved 3 documents from vector store]
[2025-09-28 09:48:26,171: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:26,763: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: relevant

[2025-09-28 09:48:26,766: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:27,885: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"]
產生內容===>: Rob 和 Jeffery 提到了與 'salary' 相關的事情。

*   **Rob** 提到 "my salary" 作為他工作中最喜歡的部分之一。
*   **Jeffery** 在後續對話中提到他 "got my salary raised from this month!" (這個月開始加薪了!)，這表明薪水是他關注的重點。

[2025-09-28 09:48:27,890: INFO: models: AFC is enabled with max remote calls: 10.]
[2025-09-28 09:48:28,450: INFO: _client: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-f